In [19]:
# Import all the libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

from pathlib import Path
from typing import Dict, Text
import warnings
warnings.filterwarnings("ignore")

## Data Loading

In [20]:
# Load tranaction data
train = pd.read_csv('Dataset/transactions_train.csv')
train = train[train['t_dat'] >='2020-09-01']

# add 0 in article_id column (string)
train['article_id'] = train['article_id'].astype(str)
train['article_id'] = train['article_id'].apply(lambda x: x.zfill(10))
train.head()

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id
17,17,2020-09-15,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007,0.061000,2
225,225,2020-09-14,0008968c0d451dbc5a9968da03196fe20051965edde741...,0893059004,0.035576,2
368,368,2020-09-20,001ea4e9c54f7e9c88811260d954edc059d596147e1cf8...,0863646005,0.033881,1
663,663,2020-09-07,003ac6eff3ca20cbe4e14a106a5e28b7551616005de8f9...,0909921001,0.025407,2
734,734,2020-09-13,004888668b9458668aa3234d17337671673599f7f84d36...,0904026002,0.033881,2


In [21]:
train['customer_id'][734]

'004888668b9458668aa3234d17337671673599f7f84d36f77fd366b5139058c5'

In [22]:
# laod customer data
customer_df = pd.read_csv('Dataset/customers.csv')
customer_df.tail(20)

,Unnamed: 0,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
9980,9980,01d8bf67bff3c69f80fa56e3e7b08eff97ef6d606d73e9...,1.0,1.0,ACTIVE,Regularly,53.0,87337617e416fe2da29e4958ced81d42a67680a04e49e3...
9981,9981,01d8c3a950508c5a6d84e9c04f3602fa9fa3539dc5bf82...,1.0,1.0,ACTIVE,Regularly,23.0,42ff5629cd0eef07585a216699fcc0b9b6b97be01e985b...
9982,9982,01d8d43eceb5bdf8ea919bb426eb34e4aba5211586d907...,1.0,1.0,ACTIVE,Regularly,38.0,dc60b8b2ba370b0f4f06165a240f175bf8dabca24281be...
9983,9983,01d8d8bd0de951df15523601dd363f1dbcb4053997b4ec...,1.0,1.0,ACTIVE,Regularly,22.0,45e52c68901dabace7723c451d817eb87430355b72ac74...
9984,9984,01d8e323bcda3d4f42b445712e00542491371c03ab6c3f...,NaN,NaN,ACTIVE,NONE,23.0,1e6787160a4e3fa62d02f4c68785a37a43993a2b9871be...
9985,9985,01d8e6a18cab3c2f3c30b574b60f74d564593a50113368...,NaN,NaN,ACTIVE,NONE,33.0,21ab3e81cc585dcc4d6966a4e537cd71536a0b7f92bec8...
9986,9986,01d8e7af4e4e937cbc66a4d88d665106dc94c3016fdad2...,NaN,NaN,PRE-CREATE,NONE,49.0,6d035a6b4d06397dee36cf82b5ec8a08f313f50d9e15cf...
9987,9987,01d8ec3f6146ad2ecaad0611c19bf3d54f192d82d9c5ab...,NaN,NaN,ACTIVE,NONE,45.0,a4bcec7334f98dd1ea42c75d05978136a39571b185d671...
9988,9988,01d8f18711392fc231760d5ae34b54f88aa3675859ffd9...,NaN,NaN,ACTIVE,NONE,19.0,448718a2c7bac635cefcfa1fb1f65f38d4df328b3fbdbb...
9989,9989,01d8f1aa43637826f2323650bf9275afd5e884d2ed9674...,NaN,NaN,ACTIVE,NONE,32.0,7306b27914f935ed027a1b53866221d2a6d127454cb2a1...


In [23]:
customer_df["customer_id"][9986]

'01d8e7af4e4e937cbc66a4d88d665106dc94c3016fdad2e7ba8aa3b75081b3b8'

In [24]:
# Laod articles data
article_df = pd.read_csv('Dataset/articles.csv')

In [25]:
# add 0 in article_id column (string) similar to train0
article_df['article_id'] = article_df['article_id'].astype(str)
article_df['article_id'] = article_df['article_id'].apply(lambda x: x.zfill(10))
article_df.tail()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
105537,0953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
105538,0953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
105539,0956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
105540,0957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.
105541,0959461001,959461,Lounge dress,265,Dress,Garment Full body,1010016,Solid,11,Off White,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,Calf-length dress in ribbed jersey made from a...


In [26]:
article_df.shape

(105542, 25)

In [27]:
#Select two features customerId and articleId for creation of embeding

unique_customer_ids = customer_df.customer_id.unique()
unique_article_ids = article_df.article_id.unique()

article_ds = tf.data.Dataset.from_tensor_slices(dict(article_df[['article_id']]))
articles = article_ds.map(lambda x: x['article_id'])

### Query tower | Candidate tower | Recommendation Model

In [28]:
embedding_dimension = 64

# Query Tower Model

customer_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_customer_ids, mask_token=None),  
  tf.keras.layers.Embedding(len(unique_customer_ids) + 1, embedding_dimension)
])

In [29]:
# Candidate Model

article_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_article_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_article_ids) + 1, embedding_dimension)
])

In [30]:
# Developing Retrieval Model

class RetrievalModel(tfrs.Model):
    
    def __init__(self, customer_model, article_model):
        super().__init__()
        self.article_model: tf.keras.Model = article_model
        self.customer_model: tf.keras.Model = customer_model
        self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=articles.batch(128).map(self.article_model),            
            ),
        )        

    def compute_loss(self, features: Dict[str, tf.Tensor], training=False) -> tf.Tensor:
    
        customer_embeddings = self.customer_model(features["customer_id"])    
        article_embeddings = self.article_model(features["article_id"])

        # The task computes the loss and the metrics.
        return self.task(customer_embeddings, article_embeddings,compute_metrics=not training)

## Training and Validation

In [31]:
model = RetrievalModel(customer_model, article_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [32]:
train = train[train['t_dat']<='2020-09-15']
test = train[train['t_dat'] >='2020-09-15']

train_ds = tf.data.Dataset.from_tensor_slices(dict(train[['customer_id','article_id']])).shuffle(100_000).batch(256).cache()
test_ds = tf.data.Dataset.from_tensor_slices(dict(test[['customer_id','article_id']])).batch(256).cache()


### Retrieval

In [33]:
# train without validation

train_ds = tf.data.Dataset.from_tensor_slices(dict(train[['customer_id','article_id']])).shuffle(100_000).batch(256).cache()

num_epochs = 50

history = model.fit(
    train_ds,    
    epochs=num_epochs,
    verbose=1)

Epoch 1/50
17/17 [==============================] - 3s 6ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 1353.0929 - regularization_loss: 0.0000e+00 - total_loss: 1353.0929
Epoch 2/50
17/17 [==============================] - 0s 3ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 1319.8479 - regularization_loss: 0.0000e+00 - total_loss: 1319.8479
Epoch 3/50
17/17 [==============================] - 0s 3ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factoriz

17/17 [==============================] - 0s 3ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 110.9712 - regularization_loss: 0.0000e+00 - total_loss: 110.9712
Epoch 20/50
17/17 [==============================] - 0s 3ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 109.5699 - regularization_loss: 0.0000e+00 - total_loss: 109.5699
Epoch 21/50
17/17 [==============================] - 0s 3ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_

17/17 [==============================] - 0s 3ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 100.4444 - regularization_loss: 0.0000e+00 - total_loss: 100.4444
Epoch 38/50
17/17 [==============================] - 0s 3ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 100.1436 - regularization_loss: 0.0000e+00 - total_loss: 100.1436
Epoch 39/50
17/17 [==============================] - 0s 3ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_

In [87]:
 a = [model.evaluate(test_ds, return_dict=True)]

1/1 [==============================] - 1s 766ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1850 - factorized_top_k/top_5_categorical_accuracy: 0.7402 - factorized_top_k/top_10_categorical_accuracy: 0.9567 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 684.6477 - regularization_loss: 0.0000e+00 - total_loss: 684.6477


In [95]:
A = pd.DataFrame(a)
#A.rename(columns = {0:'Parameters'}, inplace=True)
A = A.T
#print(A)
A.columns = ['Parameters']
print(A)
A.to_csv("C:/Users/Sushanth S/Class Lectures/Marketing/HM-Recommender-System-App-main/Project/Dataset/Twotower_perf.csv")

                                               Parameters
factorized_top_k/top_1_categorical_accuracy      0.185039
factorized_top_k/top_5_categorical_accuracy      0.740157
factorized_top_k/top_10_categorical_accuracy     0.956693
factorized_top_k/top_50_categorical_accuracy     1.000000
factorized_top_k/top_100_categorical_accuracy    1.000000
loss                                           684.647705
regularization_loss                              0.000000
total_loss                                     684.647705


## Making Predictions

In [35]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.customer_model, k=20)

In [36]:
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((articles.batch(128), articles.batch(128).map(model.article_model))))

In [37]:
# Get recommendations.
_, titles = index(tf.constant(["00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657"]))
print(f"Recommendations for user: {titles[0, :15]}")

Recommendations for user: [b'0568601043' b'0889460001' b'0763037001' b'0766346011' b'0762846027'
 b'0898596003' b'0889669007' b'0895610005' b'0898596007' b'0762846001'
 b'0806131012' b'0871274001' b'0907527001' b'0893059005' b'0910132002']


In [38]:
# Export the query model.
import os
tmp = os.environ.get("DATA_FOLDER", "/workspace/data/tensorflow")


path = "C:/Users/Sushanth S/Class Lectures/Marketing/HM-Recommender-System-App-main/Project/Models"

# Save the index.
tf.saved_model.save(index, path)

# Load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load(path)

# Pass a user id in, get top predicted movie titles back.
_, titles = loaded(['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657'])

INFO:tensorflow:Assets written to: C:/Users/Sushanth S/Class Lectures/Marketing/HM-Recommender-System-App-main/Project/Models\assets


INFO:tensorflow:Assets written to: C:/Users/Sushanth S/Class Lectures/Marketing/HM-Recommender-System-App-main/Project/Models\assets


In [39]:
print(f"Recommendations: {titles[0][:1]}")

Recommendations: [b'0568601043']


In [40]:
import pandas as pd

In [41]:
titles

<tf.Tensor: shape=(1, 20), dtype=string, numpy=
array([[b'0568601043', b'0889460001', b'0763037001', b'0766346011',
        b'0762846027', b'0898596003', b'0889669007', b'0895610005',
        b'0898596007', b'0762846001', b'0806131012', b'0871274001',
        b'0907527001', b'0893059005', b'0910132002', b'0923758001',
        b'0763275001', b'0570003002', b'0564309022', b'0929695001']],
      dtype=object)>

In [42]:
titles[0][:]

<tf.Tensor: shape=(20,), dtype=string, numpy=
array([b'0568601043', b'0889460001', b'0763037001', b'0766346011',
       b'0762846027', b'0898596003', b'0889669007', b'0895610005',
       b'0898596007', b'0762846001', b'0806131012', b'0871274001',
       b'0907527001', b'0893059005', b'0910132002', b'0923758001',
       b'0763275001', b'0570003002', b'0564309022', b'0929695001'],
      dtype=object)>

In [43]:
df = pd.DataFrame(titles.numpy()).astype("int").T

In [44]:
df.rename(columns = {0:'customer_id'}, inplace = True)

In [45]:
df['customer_id'] =df['customer_id'].astype("str")
df['customer_id'] =df['customer_id'].apply(lambda x: x.zfill(10))

In [46]:
df

,customer_id
0,0568601043
1,0889460001
2,0763037001
3,0766346011
4,0762846027
5,0898596003
6,0889669007
7,0895610005
8,0898596007
9,0762846001


In [47]:
df1 = pd.merge(train,customer_df,how="left")

In [48]:
df2 = pd.merge(article_df,df1,how="left")

In [49]:
df3 = df2.drop(['product_code','product_type_no','graphical_appearance_no',
               'graphical_appearance_name', 'colour_group_code', 
               'colour_group_name','perceived_colour_value_id', 
               'perceived_colour_value_name',
               'perceived_colour_master_id', 
               'perceived_colour_master_name',
               'department_no', 'department_name',
               'index_code', 'index_name',
               'index_group_no', 'section_no', 'section_name',
               'garment_group_no', 'detail_desc', 'Unnamed: 0',
               't_dat', 'price', 'sales_channel_id',
               'FN', 'Active',
               'club_member_status', 'fashion_news_frequency', 'age', 
               'postal_code'],axis=1)

In [50]:
df3.head()

,article_id,prod_name,product_type_name,product_group_name,index_group_name,garment_group_name,customer_id
0,0108775015,Strap top,Vest top,Garment Upper body,Ladieswear,Jersey Basic,NaN
1,0108775044,Strap top,Vest top,Garment Upper body,Ladieswear,Jersey Basic,NaN
2,0108775051,Strap top (1),Vest top,Garment Upper body,Ladieswear,Jersey Basic,NaN
3,0110065001,OP T-shirt (Idro),Bra,Underwear,Ladieswear,"Under-, Nightwear",NaN
4,0110065002,OP T-shirt (Idro),Bra,Underwear,Ladieswear,"Under-, Nightwear",NaN


In [51]:
df_chk = pd.read_csv("C:/Users/Sushanth S/Class Lectures/Marketing/HM-Recommender-System-App-main/Project/Dataset/combine_customer_matrixr.csv")
df_chk

,Unnamed: 0,article_id,prod_name,product_type_name,product_group_name,index_group_name,garment_group_name,customer_id
0,0,108775015,Strap top,Vest top,Garment Upper body,Ladieswear,Jersey Basic,NaN
1,1,108775044,Strap top,Vest top,Garment Upper body,Ladieswear,Jersey Basic,NaN
2,2,108775051,Strap top (1),Vest top,Garment Upper body,Ladieswear,Jersey Basic,NaN
3,3,110065001,OP T-shirt (Idro),Bra,Underwear,Ladieswear,"Under-, Nightwear",NaN
4,4,110065002,OP T-shirt (Idro),Bra,Underwear,Ladieswear,"Under-, Nightwear",NaN
...,...,...,...,...,...,...,...,...
107302,107302,953450001,5pk regular Placement1,Socks,Socks & Tights,Menswear,Socks and Tights,NaN
107303,107303,953763001,SPORT Malaga tank,Vest top,Garment Upper body,Ladieswear,Jersey Fancy,NaN
107304,107304,956217002,Cartwheel dress,Dress,Garment Full body,Ladieswear,Jersey Fancy,NaN
107305,107305,957375001,CLAIRE HAIR CLAW,Hair clip,Accessories,Divided,Accessories,NaN


In [52]:
# Save the `combine_customer_matrix` as a new csv

df3.to_csv('C:/Users/Sushanth S/Class Lectures/Marketing/HM-Recommender-System-App-main/Project/Dataset/combine_customer_matrixr.csv')

In [53]:
lst = df.customer_id.values.tolist()
lst
B = set(df3.loc[df3['article_id'].isin(lst)].index)
print(len(B))

45


In [54]:
df2

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,t_dat,customer_id,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107302,0953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107303,0953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107304,0956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107305,0957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
A = set(df2.loc[df2['customer_id'] == "0008968c0d451dbc5a9968da03196fe20051965edde7413775c4eb3be9abe9c2"].index)

In [56]:
A

{102096}

In [57]:
lst = df.customer_id.values.tolist()

In [58]:
B = set(df2.loc[df2['article_id'].isin(lst)].index)

In [59]:
len(B)

45

In [60]:
C = B - A

In [61]:
df2.loc[C].drop_duplicates('article_id').set_index('article_id').drop(['product_code','product_type_no',
                                                                       'graphical_appearance_no',
                                                                       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
                                                                       'perceived_colour_value_id', 'perceived_colour_value_name',
                                                                       'perceived_colour_master_id', 'perceived_colour_master_name',
                                                                       'department_no', 'department_name', 'index_code', 'index_name',
                                                                       'index_group_no', 'section_no', 'section_name',
                                                                       'garment_group_no', 'detail_desc', 'Unnamed: 0',
                                                                       't_dat', 'customer_id', 'price', 'sales_channel_id', 'FN', 'Active',
                                                                       'club_member_status', 'fashion_news_frequency', 'age', 'postal_code'],axis=1).reset_index()

,article_id,prod_name,product_type_name,product_group_name,index_group_name,garment_group_name
0,0763037001,River,Blouse,Garment Upper body,Ladieswear,Blouses
1,0889460001,Tina Blouse,Blouse,Garment Upper body,Ladieswear,Blouses
2,0898596003,Skinny Ankle RW Brooklyn,Trousers,Garment Lower body,Ladieswear,Trousers
3,0898596007,Skinny Ankle RW Brooklyn,Trousers,Garment Lower body,Ladieswear,Trousers
4,0568601043,Mariette Blazer,Blazer,Garment Upper body,Ladieswear,Dressed
5,0762846001,Lucy blouse,Shirt,Garment Upper body,Ladieswear,Blouses
6,0762846027,Lucy blouse,Shirt,Garment Upper body,Ladieswear,Blouses
7,0929695001,Hemp Denim,Blouse,Garment Upper body,Ladieswear,Blouses
8,0889669007,Apollo barell leg trouser,Trousers,Garment Lower body,Ladieswear,Trousers
9,0763275001,RYAN REG VN T-SHIRT,T-shirt,Garment Upper body,Menswear,Jersey Basic


In [64]:
def customer_article_recommend():
    
    user = input('CustomerID: ')
    n_recs = int(input('How many recommendations? '))
    df = pd.read_csv("Dataset/combine_customer_matrixr.csv")
    df.drop(['Unnamed: 0'],axis=1,inplace=True)
    df.reset_index()
    df['article_id'] =df['article_id'].astype("str")
    df['article_id'] =df['article_id'].apply(lambda x: x.zfill(10))
    # Pass a user id in, get top predicted movie titles back.
    _, titles = loaded([user])
    #titles = titles[0][:n_recs]
    #print(titles)   
    
    df1 = pd.DataFrame(titles.numpy()).astype("int").T
    #print(df1)
    df1.rename(columns = {0:'article_id'}, inplace = True)
    df1['article_id'] =df1['article_id'].astype("str")
    df1['article_id'] =df1['article_id'].apply(lambda x: x.zfill(10))
    lst = df1.article_id.values.tolist()
    #print(lst)
    A = set(df.loc[df['customer_id'] == user].index)
    #print(A)
    B = set(df.loc[df['article_id'].isin(lst)].index)
    #print(len(B))
    C = B - A
    D = df.loc[C].drop_duplicates('article_id').set_index('article_id').drop(["customer_id"],axis=1).reset_index()
    #print(type(D))
    
    return D.head(n_recs)

In [65]:
example = customer_article_recommend()

example

CustomerID: 007e4c2e0994b4aac97c2713d5dc0b7497d74113f32446868ca22b6437e9594d
How many recommendations? 5


,article_id,prod_name,product_type_name,product_group_name,index_group_name,garment_group_name
0,0861027001,AK Kanna skirt HW,Skirt,Garment Lower body,Divided,Trousers
1,0916058001,Bumling oversized joggers,Trousers,Garment Lower body,Divided,Jersey Basic
2,0897679002,Vigo Velour Sweater,Sweater,Garment Upper body,Divided,Knitwear
3,0788575004,Maja cargo Slim HW Denim,Trousers,Garment Lower body,Divided,Trousers
4,0840947002,Pauline,Skirt,Garment Lower body,Ladieswear,Skirts


In [69]:
int(example.article_id[0])

861027001